In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
from test_utils import test
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will use a toy dataset from sklearn.

In [2]:
from sklearn import datasets

#Load dataset
sklearn_dataset = datasets.load_breast_cancer()
Xx  = sklearn_dataset.data
y = sklearn_dataset.target * 2 - 1    # labels must be in {-1, 1} for the hinge loss
X = np.ones((Xx.shape[0], Xx.shape[1] + 1 ))   
X[:, :-1] = Xx
print("(N, D) =", X.shape)

(N, D) = (569, 31)


## Prepare cost and prediction functions

In [26]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective, equation (1) in the exercise pdf),
        that is loss plus regularizer.
        
    Args:
        X: the full dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)
    
    Returns:
        scalar, non-negative
        
    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_primal_objective(y_test, x_test, w_test, 1)
    49.0
    """
    return np.sum(np.maximum(0, 1-y * (X @ w))) + lambda_ / 2 * w.T @ w

In [28]:
test(calculate_primal_objective)

✅ Your `calculate_primal_objective` passed 4 tests.


In [31]:
def calculate_accuracy(y, X, w):
    """compute the accuracy on the given dataset (X, y) using the model w.
    
    Args:
        X: the full dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)
    
    Returns:
        scalar, between 0 and 1
        
    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_accuracy(y_test, x_test, w_test)
    0.5
    """
    pred = X @ w
    pred[pred < 0] = -1
    pred[pred >= 0] = 1
    return np.sum(pred == y) / len(y)

In [32]:
test(calculate_accuracy)

✅ Your `calculate_accuracy` passed 4 tests.


## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [41]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    
    Args:
        X: the dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)
        lambda_: positive scalar number
        n: the index of the (one) datapoint we have sampled
        num_examples: N
    
    Returns:
        numpy array, shape = (num_features)
    
    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_stochastic_gradient(y_test, x_test, w_test, 1, 1, 2)
    array([ 8, 11, 17])
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!

    grad = lambda_ * w
    x_n, y_n = X[n], y[n]
    if 1 - y_n * x_n.T @ w > 0:
        grad += num_examples * -y_n * x_n
    
    return grad


In [42]:
test(calculate_stochastic_gradient)

✅ Your `calculate_stochastic_gradient` passed 4 tests.


Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [43]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 2 * int(1e5)
    gamma = 1e-4
    lambda_ = int(1e4)   # big because scales with N due to the formulation of the problem (not an averaged loss)

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma / (it+1) * grad   
        
        if it % 20000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
            
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))


sgd_for_svm_demo(y, X)

iteration=0, cost=20808570.526844647
iteration=20000, cost=238.20925791122792
iteration=40000, cost=147.5698884111824
iteration=60000, cost=191.14152149712902
iteration=80000, cost=131.97491677913854
iteration=100000, cost=127.7628536717259
iteration=120000, cost=128.2848196535933
iteration=140000, cost=126.08088892639279
iteration=160000, cost=124.77013718178307
iteration=180000, cost=133.12889614028242
training accuracy = 0.9279437609841827


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [45]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    
    Args:
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        X: the dataset matrix, shape = (num_examples, num_features)
        lambda_: positive scalar number
        alpha: vector of dual coordinates, shape = (num_examples)
        w: vector of primal parameters, shape = (num_features)
        n: the coordinate to be updated
        
    Returns:
        w: updated vector of primal parameters, shape = (num_features)
        alpha: updated vector of dual parameters, shape = (num_examples)
        
    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1., 2., 3.], [4., 5., 6.]])
    >>> w_test = np.array([-0.3, -0.3, -0.3])
    >>> alpha_test = np.array([.1, .1])
    >>> calculate_coordinate_update(y_test, x_test, 1, alpha_test, w_test, 0)
    (array([-0.1,  0.1,  0.3]), array([0.3, 0.1]))
    """        
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    gamma_0 = lambda_ * (1 - y_n * w.T @ x_n) / (x_n.T @ x_n)
    gamma_opt = min(max(gamma_0 + old_alpha_n, 0), 1) - old_alpha_n

    alpha[n] += gamma_opt
    w = w + gamma_opt / lambda_ * y_n * x_n

    return w, alpha

In [46]:
test(calculate_coordinate_update)

✅ Your `calculate_coordinate_update` passed 5 tests.


In [53]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem.
    
    Args:
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        X: the dataset matrix, shape = (num_examples, num_features)
        w: vector of primal parameters, shape = (num_features)
        alpha: vector of dual coordinates, shape = (num_examples)
        lambda_: non negative scalar number

    Output:
        scalar

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1., 2., 3.], [4., 5., 6.]])
    >>> w_test = np.array([-0.3, -0.3, -0.3])
    >>> alpha_test = np.array([.1, .1])
    >>> calculate_dual_objective(y_test, x_test, w_test, alpha_test, 1)
    0.065
    """
    
    return np.sum(alpha) - w.T @ w * lambda_ / 2

In [54]:
test(calculate_dual_objective)

✅ Your `calculate_dual_objective` passed 5 tests.


In [55]:
# Notice that the gap is going to 0
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 2*int(1e5)
    lambda_ = int(1e4)

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 20000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:746.93167, dual:0.00896, gap:746.92271
iteration=20000, primal:139.40006, dual:56.74702, gap:82.65304
iteration=40000, primal:246.85227, dual:87.38888, gap:159.46340
iteration=60000, primal:125.83601, dual:103.68967, gap:22.14633
iteration=80000, primal:174.19124, dual:111.01067, gap:63.18057
iteration=100000, primal:125.81881, dual:115.06415, gap:10.75466
iteration=120000, primal:124.11814, dual:117.52024, gap:6.59790
iteration=140000, primal:124.72951, dual:119.24190, gap:5.48761
iteration=160000, primal:125.17228, dual:120.31548, gap:4.85680
iteration=180000, primal:123.09739, dual:120.96351, gap:2.13388
training accuracy = 0.9261862917398945
